<a href="https://colab.research.google.com/github/vincentmartin/tp-fine-tuning-student-version/blob/main/tp-fine-tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP fine-tuning de LLM

Dans ce notebook vous allez fine tuner un LLM de base, Flan T5, avec la technique PEFT et LoRA.

### Instruction à suivre pour exécution sur Google Colab

Aller dans `Execution -> Modifier le type d'exécution` puis sélectionner `T4-GPU` pour exploiter les fonctionnalités GPU.

![Colab GPU](resources/colab_gpu.png "T4-GPU")

Installationd des dépendances

In [ ]:
%pip install -U datasets

%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch \
    torchdata --quiet

%pip install \
    transformers \
    evaluate \
    rouge_score \
    loralib \
    peft \
    bitsandbytes

Import des dépendances

In [ ]:
from datasets import load_dataset
from transformers import AutoModel, AutoModelForCausalLM, AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer, BitsAndBytesConfig
import torch
import time
import evaluate
import pandas as pd
import numpy as np
import os
import bitsandbytes
os.environ["WANDB_DISABLED"] = "true"

Chargement du LLM de base.

In [ ]:
model_name='google/flan-t5-base'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Création d'une fonction pour afficher le nombre de paramètres entraînables.

In [ ]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(original_model))

trainable model parameters: 247577856
all model parameters: 247577856
percentage of trainable model parameters: 100.00%


## Fine tuning avec PEFT et LoRA

Le fine tuning complet d'un modèle n'est pas un choix judicieux pour un particulier ou une entreprise qui n'a pas une énorme puissance de calcul. La méthode la plus appropriée est d'utiliser PEFT (_Parameter Efficient Fine-Tuning_).

PEFT est un ensemble de technique qui incluant LORA (_Low Rank Adaptation_) et le _prompt tuning_ (**différent du prompt engineering**). LORA permet de fine tuner un modèle avec peu de ressources matérielles (un ou deux GPU). LORA permet de créer des adapteurs composés de 1-10% des paramètres du LLM original. De plus, le LLM original n'est pas modifié, ce qui permet de rapidement changer d'adapteurs en fonction du cas d'usage.

### Configuration de PEFT / LoRA

Premièrement, configurons PEFT/LoRA pour fine tuner notre modèle de base avec ce que l'on appelle _adapteur_.

PEFT/LoRA gêle les couches du LLM original pour n'entraîner que l'adapteur.

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=32, # Rank : plus il est grand, plus il y a de paramètres. Idéal : 16-32
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM # Pour FLANT5, laisser ce type
)

Ajouter l'adapteur au LLM original.

In [ ]:
peft_model = get_peft_model(original_model,
                            lora_config)
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 3538944
all model parameters: 251116800
percentage of trainable model parameters: 1.41%


## Lancement de l'entraînement

Chargeons le jeu de données pour l'entraînement.

In [ ]:
huggingface_dataset_name = "knkarthick/dialogsum"
dataset = load_dataset(huggingface_dataset_name)

def tokenize_function(example):
    start_prompt = 'Summarize the following conversation.\n\n'
    end_prompt = '\n\nSummary: '
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example["dialogue"]]
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example["summary"], padding="max_length", truncation=True, return_tensors="pt").input_ids

    return example

# The dataset actually contains 3 diff splits: train, validation, test.
# The tokenize_function code is handling all data across all splits in batches.
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['id', 'topic', 'dialogue', 'summary',])

Map:   0%|          | 0/12460 [00:00<?, ? examples/s]

Pour que l'entraînement prenne un temps acceptable dans ce notebook, nous diminuons la taille du jeu de données.

In [ ]:
tokenized_datasets = tokenized_datasets.filter(lambda example, index: index % 100 == 0, with_indices=True)

Filter:   0%|          | 0/12460 [00:00<?, ? examples/s]

Filter:   0%|          | 0/500 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1500 [00:00<?, ? examples/s]

**Exercice**  : en vous aidant de la documentation https://huggingface.co/docs/transformers/v4.15.0/en/main_classes/trainer#transformers.TrainingArguments, créer une instance de **Trainer** pour entraîner le LLM. Vous utiliserez les paramètres suivants :
- auto_find_batch_size=True,
- learning_rate=1e-3,
- num_train_epochs=5,
- logging_steps=1,
- max_steps=1   

Le jeu de données à utiliser pour l'entraînement est `tokenized_datasets["train"]`.

**Dans Google Colab, utiliser `report_to=None` sinon il vous sera demandé une clef Wanadb.**

In [ ]:
output_dir = './training-output'



peft_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    learning_rate=1e-3,
    num_train_epochs=5,
    logging_steps=1,
    max_steps=1,
    report_to=None
)

peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_datasets["train"]
)

#peft_trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


**Exercice** : Lancer l'entraînement et sauvegarder le modèle (adapteur)  et le tokenizer dans le dossier `training-output-checkpoint`.

In [ ]:
checkpoint_dir = './training-output-checkpoint'
peft_trainer.train()

# Sauvegarder le modèle et le tokenizer
peft_model.save_pretrained(checkpoint_dir)       # Sauvegarder
tokenizer.save_pretrained(checkpoint_dir)

#loaded_model = AutoModel.from_pretrained(checkpoint_dir)
#loaded_tokenizer = AutoTokenizer.from_pretrained(checkpoint_dir)

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
1,49.250000


('./training-output-checkpoint/tokenizer_config.json',
 './training-output-checkpoint/special_tokens_map.json',
 './training-output-checkpoint/spiece.model',
 './training-output-checkpoint/added_tokens.json',
 './training-output-checkpoint/tokenizer.json')

### Evaluation du modèle fine tuné

Une erreur classique lorsque l'on début est d'évaluer les performances en 'regardant' quelques générations manuellement. C'est une mauvaise idée car (1) ce n'est pas quantifié et (2) ce qui fonctionne sur quelques exemples ne fonctionne peut être pas sur des milliers d'exemples (principe de généralisation).

Lorsque l'on fine tune un modèle, il est donc capital de mesurer les performances pour savoir si **globalement** les résultats sont meilleurs.

In [ ]:
from peft import PeftModel, PeftConfig

peft_model_base = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base", torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

peft_model = PeftModel.from_pretrained(peft_model_base,
                                       'training-output-checkpoint',
                                       torch_dtype=torch.bfloat16,
                                       is_trainable=False)


In [ ]:
index = 200
dialogue = dataset['test'][index]['dialogue']
human_baseline_summary = dataset['test'][index]['summary']

device = "cuda" if torch.cuda.is_available() else "cpu"


prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """

input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

original_model_outputs = original_model.to(device).generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)


peft_model_outputs = peft_model.to(device).generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'RESUME HUMAIN:\n{human_baseline_summary}')
print(dash_line)
print(f'RESUME AVEC MODELE ORIGINAL:\n{original_model_text_output}')
print(dash_line)
print(dash_line)
print(f'RESUME AVEC MODELE PEFT: {peft_model_text_output}')

---------------------------------------------------------------------------------------------------
RESUME HUMAIN:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
---------------------------------------------------------------------------------------------------
RESUME AVEC MODELE ORIGINAL:
Is there anything you need?
---------------------------------------------------------------------------------------------------
---------------------------------------------------------------------------------------------------
RESUME AVEC MODELE PEFT: #Person1#: I'm thinking of upgrading my computer.


Inférence sur 10 exemples du jeu de test.

In [ ]:
dialogues = dataset['test'][0:10]['dialogue']
human_baseline_summaries = dataset['test'][0:10]['summary']

original_model_summaries = []
instruct_model_summaries = []
peft_model_summaries = []

for idx, dialogue in enumerate(dialogues):
    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

    human_baseline_text_output = human_baseline_summaries[idx]

    original_model_outputs = original_model.to(device).generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

    peft_model_outputs = peft_model.to(device).generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)
    original_model_summaries.append(original_model_text_output)
    peft_model_summaries.append(peft_model_text_output)

zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, peft_model_summaries))

df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_summaries', 'original_model_summaries', 'peft_model_summaries'])

**Exercice** : en utilisant la documentation https://huggingface.co/docs/evaluate/main/en/choosing_a_metric, calculer le score ROUGE entre :
- Les résumés du modèle original  (`original_model_summaries`)  vs. résumés humain (`human_baseline_summaries`).
- Les résumés du modèle peft  (`peft_model_summaries`) vs. résumé humain (`human_baseline_summaries`).

Afficher les scores et commentez les.

In [ ]:
rouge_metric = evaluate.load("rouge")

# Calculer ROUGE entre original_model_summaries et human_baseline_summaries
original_vs_human = rouge_metric.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries
)

# Calculer ROUGE entre peft_model_summaries et human_baseline_summaries
peft_vs_human = rouge_metric.compute(
    predictions=peft_model_summaries,
    references=human_baseline_summaries
)

# Afficher les scores
print("Scores ROUGE entre le modèle original et les résumés humains:")
print(original_vs_human)

print("\nScores ROUGE entre le modèle PEFT et les résumés humains:")
print(peft_vs_human)

Scores ROUGE entre le modèle original et les résumés humains:
{'rouge1': 0.24138795803928403, 'rouge2': 0.06766815310293572, 'rougeL': 0.19967870226360313, 'rougeLsum': 0.2012315304848285}

Scores ROUGE entre le modèle PEFT et les résumés humains:
{'rouge1': 0.23308938746438745, 'rouge2': 0.10011594202898552, 'rougeL': 0.21507407407407406, 'rougeLsum': 0.21857621082621081}


**On peut conclure que le modèle a appris, parce que sous PEFT modèle le score ROUGE est un peu plus elévé**

**Exercice** : calculer le gain de performance en pourcentage du modèle PEFT sur le modèle original

In [ ]:
gain_rouge1 = ((peft_vs_human['rouge1'] - original_vs_human['rouge1']) / original_vs_human['rouge1']) * 100
gain_rouge2 = ((peft_vs_human['rouge2'] - original_vs_human['rouge2']) / original_vs_human['rouge2']) * 100
gain_rougeL = ((peft_vs_human['rougeL'] - original_vs_human['rougeL']) / original_vs_human['rougeL']) * 100

# Affichage
print("Gain de performance du modèle PEFT sur le modèle original (en %):")
print(f"ROUGE-1 : {gain_rouge1:.2f}%")
print(f"ROUGE-2 : {gain_rouge2:.2f}%")
print(f"ROUGE-L : {gain_rougeL:.2f}%")

Gain de performance du modèle PEFT sur le modèle original (en %):
ROUGE-1 : -3.44%
ROUGE-2 : 47.95%
ROUGE-L : 7.71%


## Exercice : Fine tuning de Llama 3

Le modèle `flan-t5-base`que nous avons utilisé jusqu'à maintenant est bien pour comprendre les principes mais c'est un modèle ancien aux performances dépassées par rapport aux modèles récents tels que Llama 3.

Dans cet exercice, vous allez charger puis fine tuner un LLM bien plus performant tout en conservant une taille acceptable de 3B de paramètres : Llama 3.2 - 3B.

Afin que le modèle puisse être chargé en VRAM, nous utiliserons une version quantisée en 4bits : https://huggingface.co/unsloth/Llama-3.2-3B-Instruct-bnb-4bit.

**Redémarrer la session à ce stade pour réinitialiser la RAM et la VRAM**

### Conseils pour réaliser l'exercice :

- Le modèle n'est plus de type _Encoder Decoder_ (Seq2Seq) mais _Decoder only_ (CausalLM). Effectuer les modiciation en conséquence
- Réduire la taille du jeu de donnée d'entraînement pour restant dans des temps acceptables (100 exemples)
- Modifier les arguments d'entraînement (`TrainingArguments`) pour prendre accélérer le traitement : considérer les paramètres `per_device_train_batch_size`, `gradient_accumulation_steps`, `gradient_chekpointing`.

L'exercice peut prendre un certain temps, faites votre maximum et avancer pas à pas.

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"
!pip install bitsandbytes --upgrade
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
import torch
import bitsandbytes as bnb

huggingface_dataset_name = "knkarthick/dialogsum"
dataset = load_dataset(huggingface_dataset_name)


model_name = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit"
output_dir = "./llama-3b-finetuned"
# Charger le modèle quantisé
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",  # Automatique pour gérer le GPU
    load_in_4bit=True,  # Charger en 4 bits
    torch_dtype=torch.float16
)
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],  # Modules to apply LoRA
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",  # Since we're using a causal language model
)
model = get_peft_model(model, lora_config)
# Charger le tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
#dataset = load_dataset("path_to_your_dataset")  # Remplacez par votre dataset
train_dataset = dataset["train"].select(range(100))
eval_dataset = dataset["validation"].select(range(100))



training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    logging_steps=10,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    learning_rate=1e-4,
    fp16=True,
    gradient_checkpointing=True,
    report_to=None
)

def preprocess_function(examples):
    inputs = [f"Instruction: {text}" for text in examples["dialogue"]]
    targets = examples["summary"]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)
    labels = tokenizer(targets, max_length=512, truncation=True).input_ids
    model_inputs["labels"] = labels
    return model_inputs

tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_eval_dataset = eval_dataset.map(preprocess_function, batched=True)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    tokenizer=tokenizer
)

trainer.train()


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-8-e008efae248e>:65: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
`use_cache=True` is incomp

ValueError: Expected input batch_size (144) to match target batch_size (21).